In [0]:
%run ../configurations/paths_config

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,DateType,StructType,StructField
from pyspark.sql.functions import col,lit
from pyspark.sql import functions as sf
import json


In [0]:
new_schema = StructType(fields=[StructField("code",StringType(),True),
                                StructField("dob",DateType(),True),
                                StructField("driverId",IntegerType(),False),
                                StructField("name",StructType()\
                                    .add("forename",StringType())\
                                    .add("surname",StringType()),True),
                                StructField("nationality",StringType(),True),
                                StructField("number",IntegerType(),True),
                                StructField("url",StringType(),True)])

In [0]:
raw_df = spark.read.schema(new_schema).json(raw_path+f"/{v_file_date}/drivers.json") 

In [0]:
trasnformed_df = raw_df.withColumn("name",sf.expr("concat(name.forename,' ',name.surname)"))

In [0]:
trimmed_df = trasnformed_df.select(col('code'),col('dob').alias('date_of_birth'),col('driverId').alias('driver_id'),col('name'),col('nationality'),col('number'))

In [0]:
audited_df = trimmed_df.withColumn("ingestion_timestamp",sf.current_timestamp())

In [0]:
audited_df.write.mode("overwrite").format("delta").saveAsTable("formula1_silver.drivers")

In [0]:
%sql
SELECT * FROM formula1_silver.drivers;